<a href="https://colab.research.google.com/github/kmk4444/Retrieval-augmented-generation/blob/main/Part7_reranking_with_cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Reranking is used to develop RAG.

In [1]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo cohere >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo streamlit >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo llama-index >> requirements.txt
!echo pypdf >> requirements.txt
!echo chromadb >> requirements.tx
!echo beautifulsoup4 >> requirements.tx
!echo matplotlib >> requirements.tx
!echo rank_bm25 >> requirements.tx
!echo replicate >> requirements.txt

In [2]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
%%writefile app.py
import streamlit as st
import cohere
import os
from dotenv import load_dotenv

#load_dotenv()
#my_key_cohere = os.getenv("cohere_apikey")
my_key_cohere="----"
#cohere_client = cohere.Client(api_key=my_key_cohere)

query = "Türkiye'nin başkenti neresidir?"

documents = [
   "Ankara, Türkiye'nin başlıca turistik lokasyonlarından biridir ve her yıl beş milyondan fazla turist ev sahipliği yapar.",
   "Genç adam Başkent Üniversitesi'nde bilgisayar mühendisliği bölümüne yerleştiği için son derece heyecanlıydı.",
   "Orta kuşak ikliminin özelliklerini gösteren Türkiye, dünya üzerinde dört mevsimi birden yaşayan şanslı coğrafyalardan biridir.",
   "Türkiye'nin başkenti Ankara'dır",
   "Yahya Kemal bir şiirinde Ankara'nın en çok İstanbul'a dönüş yolunu sevdim diyordu ama o sıralar eski bir başkent olarak bugünkü kaotik metropol havasından uzaktı.",
   "Cumhuriyetin ilanı ile birlikte yaşanan önemli değişikliklerden biri de başkentin yer değiştirerek Ankara'ya taşınmış olmasıydı.",
   "Başkentin neresi olması gerektiğiyle ilgili tartışmalar sürerken, güvenli ve merkezi bir lokasyon aranıyordu tıpkı Ankara gibi.",
   "Elektromanyetizmayla ilgili çalışmalarıyla ünlenen Maxwell, Einstein gibi pek çok bilim adamına da ilham olmuştu.",
   "Ankara'nın başlıca ilçeleri Çankaya, Yenimahalle ve Keçiören'dir diyebiliriz.",
   "Kentleşme, ülkemizde geç başlamış ve sancıları halen devam etmekte olan bir sosyolojik süreçtir.",
   "Başkent Ankara'da yılın ilk kar yağışının keyfini yine çocuklar çıkardı.",
   "Türkiye Cumhuriyeti devletinin resmi başkenti olan Ankara şehri İç Anadolu bölgesinde yer alır."
   ]

st.set_page_config(layout="wide")
st.title("Advanced RAG: Reranking | Yeniden Sıralama")
st.divider()

col_left, col_input, col_right = st.columns([1,8,1])

with col_left:
  st.empty()

with col_input:
  submit_btn = st.button(label="Yeniden Sırala", use_container_width=True)

with col_right:
  st.empty()

col_original, col_dummy, col_reranked = st.columns([9,1,9])

with col_original:
  st.subheader("Orijinal Sırayla Dokümanlar")
  for doc in documents:
    st.info(doc)

with col_dummy:
  st.empty()

with col_rekanked:
  st.subheader("Yeniden Sıralanmış Dokümanlar")

if submit_btn:
  results = cohere_client.rerank(query=query, documents=documents, top_n=12, model="rerank-multilingual-v2.0")

  for result in results:
    col_reranked.success(f"{result.document['text']} || #Sıra {result.index+1}")

Writing app.py


In [4]:
!npm install localtunnel
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.671s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 2.624s
your url is: https://slick-gifts-prove.loca.lt
^C
